In [1]:
%load_ext autoreload 
%autoreload 2

In [11]:
import os 
import sys 
import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl
import wandb
import numpy as np 
import matplotlib.pyplot as plt
from lightning.pytorch.loggers import WandbLogger


sys.path.append('/Users/ivan_zorin/Documents/DEV/code/ntl/')

from models import *
from data.data import SGCCDataset, sgcc_train_test_split
from utils.utils import *

from tqdm.auto import tqdm 
import models

In [18]:
config = load_config('../configs/config.yaml', '../configs/local_pathes.yaml')
config

namespace(logger={'name': 'first-run',
                  'notes': '',
                  'project': 'ntl',
                  'offline': False,
                  'save_dir': '/Users/ivan_zorin/Documents/DEV/runs/'},
          num_workers=2,
          n_epochs=100,
          batch_size=8,
          device='cpu',
          supervised_validation=True,
          val_every_n_steps=5,
          model='LSTMAE_old',
          model_kwargs={'input_size': 1, 'hidden_size': [64], 'n_lstms': 1},
          loss='l1_loss',
          optimizer='Adam',
          optimizer_kwargs={'lr': 0.001},
          scheduler=None,
          pca_dim=2,
          ntl_wd='/Users/ivan_zorin/Documents/DEV/code/ntl/',
          data_path='/Users/ivan_zorin/Documents/DEV/data/sgcc/data.csv',
          save_dir='/Users/ivan_zorin/Documents/DEV/runs/')

In [21]:
wandb_logger = WandbLogger(
    name='first-run',
    project='NTL',
    config=config.__dict__,
    offline=True
)

wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id 0jr3azbm.


In [20]:
wandb.finish()

In [4]:
train_data, val_data, test_data = sgcc_train_test_split(config)

num_workers = 1
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_data, batch_size=config.batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False,  num_workers=num_workers)

In [6]:
lstmae = LSTMAE_old(**config.model_kwargs)

In [13]:
model = LitLSTMAE(
    model=lstmae,
    loss_fn=F.l1_loss,
    optimizer=torch.optim.Adam,
    config=config
    
)

In [14]:
trainer = pl.Trainer(fast_dev_run=False,
                     check_val_every_n_epoch=1,
                     limit_train_batches=10,
                     limit_val_batches=10,
                     max_epochs=2,
                    #  num_sanity_val_steps=1,
                     accelerator=config.device,
                    #  profiler='simple',
                     logger=wandb_logger,
                     log_every_n_steps=1,
)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ivan_zorin/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [16]:
trainer.train_dataloader

In [ ]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
    
)


In [ ]:

wandb.finish()